In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2_contingency, spearmanr, skew, pearsonr

***Bộ dữ liệu transaction chứa giao dịch, identity chứa các thông tin***

In [3]:
df_train = pd.read_csv('/content/drive/MyDrive/Datasets/ieee-fraud-detection/default datasets/train_transaction.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Datasets/ieee-fraud-detection/default datasets/train_identity.csv')
print(df_train.shape)
print(df_test.shape)

(590540, 394)
(144233, 41)


***Tiến hành hợp nhất 2 bộ dữ liệu này lại; Output của bộ dữ liệu này là 'isFraud'***

In [4]:
data = pd.merge(df_train, df_test, how='inner', left_on=['TransactionID'], right_on=['TransactionID'])
data.shape

(144233, 434)

# **CHECKING**

In [5]:
remove_null_cols = []
for i in data.columns:
  if data[i].isnull().sum() > int(data.shape[0]/3):
    remove_null_cols.append(i)
len(remove_null_cols)

187

In [6]:
data = data.drop(columns = remove_null_cols)

In [7]:
null_cols = []
for i in data.columns:
  if data[i].isnull().sum() > 0:
    print(i, data[i].isnull().sum())
    null_cols.append(i)
  else:
    continue

card2 902
card3 172
card4 184
card5 956
card6 178
P_emaildomain 13391
R_emaildomain 13150
D1 218
V95 64
V96 64
V97 64
V98 64
V99 64
V100 64
V101 64
V102 64
V103 64
V104 64
V105 64
V106 64
V107 64
V108 64
V109 64
V110 64
V111 64
V112 64
V113 64
V114 64
V115 64
V116 64
V117 64
V118 64
V119 64
V120 64
V121 64
V122 64
V123 64
V124 64
V125 64
V126 64
V127 64
V128 64
V129 64
V130 64
V131 64
V132 64
V133 64
V134 64
V135 64
V136 64
V137 64
V167 4602
V168 4602
V169 4414
V170 4414
V171 4414
V172 4602
V173 4602
V174 4414
V175 4414
V176 4602
V177 4602
V178 4602
V179 4602
V180 4414
V181 4602
V182 4602
V183 4602
V184 4414
V185 4414
V186 4602
V187 4602
V188 4414
V189 4414
V190 4602
V191 4602
V192 4602
V193 4602
V194 4414
V195 4414
V196 4602
V197 4414
V198 4414
V199 4602
V200 4414
V201 4414
V202 4602
V203 4602
V204 4602
V205 4602
V206 4602
V207 4602
V208 4414
V209 4414
V210 4414
V211 4602
V212 4602
V213 4602
V214 4602
V215 4602
V216 4602
V217 13964
V218 13964
V219 13964
V220 3127
V221 3127
V222 3127
V

***Xử lý các giá trị Null ở các columns còn lại:***

In [9]:
data['card1'] = data['card1'].astype(np.float64)

In [10]:
null_category_cols = []
null_numeric_cols = []

# Giả sử null_cols đã được xác định từ trước
for i in null_cols:
    if data[i].dtype == 'object':
        null_category_cols.append(i)
    elif len(data[i].unique()) < 15:
        null_category_cols.append(i)
    else:
        null_numeric_cols.append(i)

print(f"Total null columns: {len(null_cols)}")
print(f"Categorical null columns: {len(null_category_cols)}")
print(f"Numeric null columns: {len(null_numeric_cols)}")

Total null columns: 225
Categorical null columns: 46
Numeric null columns: 179


***Xử lý các giá trị Null ở các columns Phân loại, ta sẽ chọn bằng giá trị chiếm đại đa số***

In [11]:
for i in null_category_cols:
  data[i] = data[i].fillna(data[i].mode()[0])

***Xử lý các giá trị Null ở các columns Liên tục bằng giá trị trung bình:***

In [12]:
for i in null_numeric_cols:
  data[i] = data[i].fillna(data[i].mean())

In [16]:
data = data.dropna()

***Vậy là dữ liệu đã hết Null***

In [23]:
data

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0,86506,50.000,H,4497.0,514.0,150.0,mastercard,102.0,...,144.0,New,NotFound,samsung browser 6.2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,0,86535,15.000,H,2803.0,100.0,150.0,visa,226.0,...,500.0,New,NotFound,mobile safari 11.0,T,F,F,T,mobile,iOS Device
2,2987010,0,86549,75.887,C,16496.0,352.0,117.0,mastercard,134.0,...,142.0,Found,Found,chrome 62.0,F,F,T,T,desktop,Windows
3,2987011,0,86555,16.495,C,4461.0,375.0,185.0,mastercard,224.0,...,507.0,New,NotFound,chrome 62.0,F,F,T,T,desktop,Windows
4,2987016,0,86620,30.000,H,1790.0,555.0,150.0,visa,226.0,...,575.0,Found,Found,chrome 62.0,T,F,T,T,desktop,MacOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,0,15810802,48.877,C,12019.0,305.0,106.0,mastercard,224.0,...,139.0,Found,Found,chrome 66.0 for android,F,F,T,F,mobile,F3111 Build/33.3.A.1.97
144229,3577526,1,15810876,250.000,R,1214.0,174.0,150.0,visa,226.0,...,333.0,New,NotFound,chrome 55.0 for android,T,F,T,F,mobile,A574BL Build/NMF26F
144230,3577529,0,15810912,73.838,C,5096.0,555.0,185.0,mastercard,137.0,...,411.0,New,NotFound,chrome 65.0 for android,F,F,T,F,mobile,Moto E (4) Plus Build/NMA26.42-152
144231,3577531,0,15810935,400.000,R,6019.0,583.0,150.0,visa,226.0,...,368.0,Found,Found,chrome 66.0,T,F,T,F,desktop,MacOS


In [24]:
#Lưu file .csv:
data.to_csv('ieee-fraud-detection-changed.csv', index=False)